In [1]:
import tensorflow as tf
from tensorflow import keras

먼저 텐서플로 데이터셋을 사용해 데이터를 적재하겠습니다.

In [2]:
import tensorflow_datasets as tfds

# with_info=True로 지정해서 데이터셋에 대한 정보도 함께 얻습니다.
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\jiweo\tensorflow_datasets\tf_flowers\3.0.1.incompleteKB0UEZ\tf_flowers-train.tfrecord*...: …

Dataset tf_flowers downloaded and prepared to C:\Users\jiweo\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.


In [3]:
dataset_size = info.splits['train'].num_examples
class_names = info.features['label'].names
n_classes = info.features['label'].num_classes

이 데이터셋에는 train 세트만 있고 테스트, 검증 세트는 없습니다.<br>
따라서 훈련 세트를 나눠야 합니다.<br>
TF 데이터셋에는 이를 위한 API가 제공됩니다.

아래 코드는 데이터셋을 테스트 세트 10%, 검증 세트 15%, 훈련 세트 75%로 나누는 코드입니다.

In [4]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

이제 이미지를 전처리해야합니다.<br>
이 CNN 모델은 224 \\(\times\\) 224 크기 이미지를 기대하므로 크기를 조정해야 합니다.<br>
또한 xception 패키지의 preprocess_input() 함수로 이미지를 전처리해야 합니다.

In [5]:
from functools import partial

def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [6]:
batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

위의 전처리 과정이 이해가 되지 않으면 아래의 전처리 함수를 사용하세요.

```py
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label
```

이제 이미지넷에서 사전훈련된 Xception 모델을 로드합니다.<br>
include_top=False로 지정하여 네트워크의 최상층에 해당하는 전역 평균 풀링 층과 밀집 출력 층을 제외시킵니다.

이 기반 모델의 출력을 바탕으로 새로운 전역 평균 풀링 층을 추가하고 그 뒤에 클래스마다 하나의 유닛과 소프트맥스 활성화 함수를 가진 밀집 출력 층을 놓고 케라스의 Model 클래스 객체를 만듭니다.

In [7]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

83697664/83683744 [==============================] - 13s 0us/step


[여기](https://justweon-dev.tistory.com/43#%EC%BC%--%EB%-D%BC%EC%-A%A-%EB%A-%BC%--%EC%-D%B-%EC%-A%A-%ED%--%-C%--%EC%A-%--%EC%-D%B-%--%ED%--%--%EC%-A%B-)에서 다룬 것처럼 훈련 초기에는 사전훈련된 층의 가중치를 동결하는 게 좋습니다.

In [8]:
for layer in base_model.layers:
    layer.trainable = False

이제 모델을 컴파일하고 훈련합니다.

In [9]:
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

Epoch 1/5
86/86 [==============================] - 118s 1s/step - loss: 1.3411 - accuracy: 0.7969 - val_loss: 1.2913 - val_accuracy: 0.8143
Epoch 2/5
86/86 [==============================] - 116s 1s/step - loss: 0.5472 - accuracy: 0.8964 - val_loss: 0.8884 - val_accuracy: 0.8529
Epoch 3/5
86/86 [==============================] - 121s 1s/step - loss: 0.3296 - accuracy: 0.9193 - val_loss: 0.8915 - val_accuracy: 0.8474
Epoch 4/5
86/86 [==============================] - 121s 1s/step - loss: 0.2465 - accuracy: 0.9400 - val_loss: 0.6847 - val_accuracy: 0.8529
Epoch 5/5
86/86 [==============================] - 117s 1s/step - loss: 0.2061 - accuracy: 0.9437 - val_loss: 0.6780 - val_accuracy: 0.8640


몇 번의 에포크동안 훈련하면 검증 정확도가 어느 수준에 도달한 후 더 나아지지 않는데, 이는 새로 추가한 최상위 층이 잘 훈련되었다는 것을 의미합니다.

이제 모든 층의 동결을 해제하고 훈련을 계속합니다.

In [10]:
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=40)

Epoch 1/40
86/86 [==============================] - 591s 7s/step - loss: 0.3312 - accuracy: 0.8895 - val_loss: 0.5570 - val_accuracy: 0.8603
Epoch 2/40
86/86 [==============================] - 595s 7s/step - loss: 0.1063 - accuracy: 0.9677 - val_loss: 0.3093 - val_accuracy: 0.9081
Epoch 3/40
86/86 [==============================] - 616s 7s/step - loss: 0.0765 - accuracy: 0.9800 - val_loss: 0.2899 - val_accuracy: 0.9191
Epoch 4/40
86/86 [==============================] - 614s 7s/step - loss: 0.0408 - accuracy: 0.9880 - val_loss: 0.2575 - val_accuracy: 0.9246
Epoch 5/40
86/86 [==============================] - 614s 7s/step - loss: 0.0380 - accuracy: 0.9876 - val_loss: 0.2724 - val_accuracy: 0.9154
Epoch 6/40
86/86 [==============================] - 575s 7s/step - loss: 0.0310 - accuracy: 0.9898 - val_loss: 0.3306 - val_accuracy: 0.9007
Epoch 7/40
86/86 [==============================] - 579s 7s/step - loss: 0.0227 - accuracy: 0.9935 - val_loss: 0.3074 - val_accuracy: 0.9228
Epoch 8/40
86

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_set)